Теория: https://habr.com/ru/company/ruvds/blog/585414/

https://github.com/oashour/NonlinearSchrodinger.jl

https://ru.wikipedia.org/wiki/Распространение_филамента

$$ \psi (x,t_0+ \Delta t) = \exp (i \Delta t \mathcal{N} /2) \mathcal{F}^{-1}\left [ \exp (-i \Delta t \alpha k^2 /2) \mathcal{F} \left [\exp(i \Delta t \mathcal{N} /2) \psi(x,t_0)\right]\right] $$

$\mathcal{N} = \sigma|\psi|^2$

In [ ]:
using Plots, FFTW

In [ ]:
# начальная волновая функция, кол-во кадров в анимации, кол-во шагов между кадрами
function nlse_ssft!( ψ, timelayers, timesteps; 
        dt = 0.05, α = 0.5, σ = 1.0, xsteps = 128, xbox = (-10, 10) )
    # шаг по времени, дисперсия, нелинейность, 
    # количество элементов  в массиве вдоль одного измерения, границы в пространстве
    
    Nd = ndims(ψ) # мерность задачи
    Nx = size(ψ,1)
    X = range(xbox[1], length = Nx, stop=xbox[2])
    T = range(dt*timesteps, length = timelayers, step = dt*timesteps)
    dx = step(X)
    
    xcut = Nx ÷ xsteps # на выходе не весь тяжелый массив
    xscaled = [xcut:xcut:Nx for _ in 1:Nd]
    Psi = Array{ComplexF64, Nd}[]
    
    p = im*dt*σ
    K = fftfreq(Nx) * (2π/dx)
    k² = [ sum(i-> K[i]^2, Tuple(i) ) for i in CartesianIndices(size(ψ)) ]
    eᵏ = exp.(-im*α*dt*k²)
    
    for i in 1:timelayers
        for _ = 1:timesteps
            ψ .*= exp.( p*abs2.(ψ) )
            fft!(ψ)
            ψ .*= eᵏ
            ifft!(ψ)
        end
        push!(Psi, ψ[xscaled...])
    end
    
    return X[xscaled[1]], T, Psi
end;

# 1D

In [ ]:
psi0 = 0.001randn(ComplexF64, 256) .+ 1.0
@time X, T, Psi = nlse_ssft!( psi0, 2^7, 2^9, dt = 5e-4,  xsteps = 128, xbox = (-8π, 8π) );

In [ ]:
Psi = hcat(Psi...);

In [ ]:
contour(T, X, real.(Psi), colorbar = true, fill = true)
xaxis!("t"); yaxis!("x")

In [ ]:
savefig("1d_soliton.png")

In [ ]:
anim = @animate for i ∈ 1:size(Psi,2)
    plot(X, real.( Psi[:,i] ), line = (3, :black), 
        fillrange = -5, fillalpha = 0.3, fillcolor = :black, ylims = (-5, 5), legend = false)
end
gif(anim, "1d_fillament.gif", fps = 10)

# 2D

In [ ]:
psi0 = 0.001randn(ComplexF64, 64, 64) .+ 1.0
@time X, T, Psi = nlse_ssft!( psi0, 2^7, 2^8, dt = 5e-4,  xsteps = 64, xbox = (-8π, 8π) );

In [ ]:
heatmap(X, X, abs2.(Psi[58]), colorbar = true)

In [ ]:
anim = @animate for i ∈ 1:size(Psi,1)
    heatmap(X, X, real.(Psi[i]), colorbar = true, title = "$i" )#, clim = (-4,4)
end
gif(anim, "2d_fillament.gif", fps = 10)

# 3D

In [ ]:
psi0 = 0.001randn(ComplexF64, 64, 64, 64) .+ 1.0
@time X, T, Psi = nlse_ssft!( psi0, 2^7, 2^7, dt = 8e-4,  xsteps = 64, xbox = (-8π, 8π) );

In [ ]:
heatmap(X, X, abs2.(Psi[80][:,:,32]) + 
            0.7abs2.(Psi[80][:,:,31]) + 
            0.3abs2.(Psi[80][:,:,30]) + 
            0.1abs2.(Psi[80][:,:,29]), colorbar = true)

In [ ]:
savefig("fillamento.png")

In [ ]:
anim = @animate for i ∈ 1:64 # томография в определенный момент времени
    heatmap(abs2.(Psi[80][:,:,i]), xticks = false, yticks = false, 
        colorbar = true ) #, size = (400,400) , clims = (20,50)
end

gif(anim, "3d_space.gif", fps = 10)

In [ ]:
anim = @animate for i ∈ 1:size(Psi,1) # оволюция определенного среза
    heatmap(abs2.(Psi[i][:,:,32]), xticks = false, yticks = false, 
        colorbar = true ) #, size = (400,400) , clims = (20,50)
end

gif(anim, "3d_fillament.gif", fps = 10)

In [ ]:
psi0 = [ soliton(x+y+z) for x in range(-2π, 2π, length = 64), y in range(-2π, 2π, length = 64),
    z in range(-2π, 2π, length = 64)]
@time X, T, Psi = nlse_ssft!( psi0, 20, 2^7, dt = 2e-5,  xsteps = 64, xbox = (-2π, 2π) );

In [ ]:
sizeof(Psi[1])/2^20 # mb

In [ ]:
contour(abs2.(Psi[20][:,:,32]), colorbar = true, fill = true)